# VGG16 without bird classes from ImageNet
# Good choice as it's simpler to understand than more complex architectures
# Will need careful memory management due to VGG16's size



# Conservative augmentations:
- Horizontal flips (birds often look similar from either side)
- Small rotation range (±15° max to maintain natural poses)
- Minimal brightness/contrast adjustments
- Small random crops while preserving main features

# Progressive training
1. First freeze VGG16 layers
2. Train only classifier layers
3. Gradually unfreeze later layers
4. Use small learning rate for fine-tuning

# Bird Species Classification Project
## BDMA 7 - Computer Vision Competition

This notebook documents the development of a bird species classification model following strict competition guidelines. We'll use a systematic approach to ensure both model performance and rule compliance.

### 1. Data Exploration and Setup

First, we'll explore our dataset structure, load necessary libraries, and analyze our class distribution. Understanding our data is crucial for making informed decisions about model architecture and training strategy.

In [ ]:
!unzip /content/bdma-07-competition.zip

In [ ]:
!ls -R

In [ ]:
import torch

# Check if CUDA is available
print(torch.cuda.is_available())

# Show CUDA device name
print(torch.cuda.get_device_name(0))

# Verify CUDA version
print(torch.version.cuda)

True
Tesla T4
12.4


In [ ]:
import torch
import sys
# Comprehensive GPU Check
def gpu_check():
    print("Python Version:", sys.version)
    print("PyTorch Version:", torch.__version__)

    try:
        print("CUDA Available:", torch.cuda.is_available())
        print("CUDA Version:", torch.version.cuda)
        print("cuDNN Version:", torch.backends.cudnn.version())
        print("Device Count:", torch.cuda.device_count())

        if torch.cuda.is_available():
            for i in range(torch.cuda.device_count()):
                print(f"GPU {i} Name:", torch.cuda.get_device_name(i))
    except Exception as e:
        print("Error:", str(e))

gpu_check()

Python Version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
PyTorch Version: 2.5.1+cu124
CUDA Available: True
CUDA Version: 12.4
cuDNN Version: 90300
Device Count: 1
GPU 0 Name: Tesla T4


### Data Analysis Insights

Let's analyze key aspects of our dataset:

1. Dataset Size:
   - Training set: 1,082 images
   - Validation set: 103 images
   - Test set: 400 images

2. Class Distribution:
   - We have 20 different bird species
   - Understanding if the dataset is balanced will inform our training strategy
   
3. Directory Structure:
   - Images are organized by species in separate folders
   - This hierarchical structure will help in creating our data loader

### 2. Data Loading and Visualization

Before building our model, it's essential to:
1. Create efficient data loading pipelines
2. Visualize sample images to understand our data
3. Set up proper data transformations

This will help us:
- Verify image quality and characteristics
- Design appropriate augmentation strategies
- Identify potential challenges in classification

In [ ]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

class BirdDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

        # Verify and clean data paths
        valid_samples = []
        for idx, row in df.iterrows():
            # Construct full path including train_images
            full_path = os.path.join(root_dir, 'train_images', row['path'])
            if os.path.exists(full_path):
                valid_samples.append(idx)
            else:
                print(f"Warning: File not found: {full_path}")

        self.df = df.iloc[valid_samples].reset_index(drop=True)
        print(f"Found {len(self.df)} valid images out of {len(df)} total")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Include train_images in the path
        img_path = os.path.join(self.root_dir, 'train_images', self.df.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        label = self.df.iloc[idx]['class_idx']

        if self.transform:
            image = self.transform(image)

        return image, label

# Basic transforms
basic_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Let's verify our directory structure
print("Current working directory:", os.getcwd())
print("\nContents of BDMA7_project_files:")
print(os.listdir('BDMA7_project_files'))

# Create dataset
train_dataset = BirdDataset(
    train_df,
    root_dir='BDMA7_project_files',
    transform=basic_transform
)

# Create dataloader
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0  # Important for debugging
)

def show_batch(dataloader, num_images=4):
    """Display a batch of images with their labels"""
    images, labels = next(iter(dataloader))
    class_names = train_df['class'].unique()

    plt.figure(figsize=(12, 6))
    for i in range(min(num_images, len(images))):
        plt.subplot(2, 2, i + 1)
        img = images[i].permute(1, 2, 0)
        # Denormalize if necessary
        img = torch.clamp(img, 0, 1)
        plt.imshow(img)
        plt.title(f'Class: {class_names[labels[i]]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Try to display images
try:
    print("\nSample images from training set:")
    show_batch(train_loader)
except Exception as e:
    print(f"Error displaying images: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
print(os.path.exists('BDMA7_project_files/train_images'))

True


### Data Loading Pipeline Analysis

Our data loading system is designed to:
1. Handle the hierarchical directory structure
2. Convert images to RGB format (handling grayscale if present)
3. Resize images to a standard size (224x224 for VGG16)

Next Steps:
1. Implement data augmentation pipeline
2. Add class mapping functionality
3. Set up validation data loader

Key Considerations:
- Memory efficiency
- Image preprocessing requirements
- Batch size optimization

### 3. Dataset Analysis and Visualization Tools

Now that we can load our images correctly, let's create tools to:
1. Analyze image properties (sizes, aspect ratios)
2. Visualize class distribution
3. Check image quality and characteristics
4. Understand typical bird poses and orientations

This information will help us make informed decisions about:
- Appropriate image resizing strategies
- Augmentation techniques
- Model architecture modifications

In [ ]:
import torch
from torchvision.utils import make_grid
import seaborn as sns

def analyze_dataset_stats():
    """Analyze and visualize dataset statistics"""
    # Class distribution
    plt.figure(figsize=(15, 5))
    class_dist = train_df['class'].value_counts()
    sns.barplot(x=class_dist.index, y=class_dist.values)
    plt.xticks(rotation=45, ha='right')
    plt.title('Class Distribution in Training Set')
    plt.xlabel('Bird Species')
    plt.ylabel('Number of Images')
    plt.tight_layout()
    plt.show()

    # Print class statistics
    print("\nClass Statistics:")
    print(f"Number of classes: {len(class_dist)}")
    print(f"Average images per class: {class_dist.mean():.1f}")
    print(f"Min images in a class: {class_dist.min()} ({class_dist.idxmin()})")
    print(f"Max images in a class: {class_dist.max()} ({class_dist.idxmax()})")

def analyze_image_properties(dataset, num_samples=100):
    """Analyze image sizes and aspect ratios"""
    sizes = []
    aspect_ratios = []

    for i in range(min(num_samples, len(dataset))):
        img_path = os.path.join('BDMA7_project_files/train_images', dataset.df.iloc[i]['path'])
        with Image.open(img_path) as img:
            w, h = img.size
            sizes.append((w, h))
            aspect_ratios.append(w/h)

    # Plot size distribution
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.scatter([s[0] for s in sizes], [s[1] for s in sizes], alpha=0.5)
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.title('Image Dimensions Distribution')

    plt.subplot(1, 2, 2)
    plt.hist(aspect_ratios, bins=20)
    plt.xlabel('Aspect Ratio (width/height)')
    plt.ylabel('Count')
    plt.title('Aspect Ratio Distribution')
    plt.tight_layout()
    plt.show()

# Run analyses
print("Analyzing dataset statistics...")
analyze_dataset_stats()

print("\nAnalyzing image properties...")
analyze_image_properties(train_dataset)

### Data Augmentation

Class Balance:


- The distribution is remarkably balanced, with all classes having between 50-60 samples
- This natural balance means we don't need to implement class weighting or oversampling techniques


Image Dimensions:


- Most images cluster around similar dimensions
- VGG16's input requirements (224x224) will standardize these variations
- The aspect ratios show most images are in a reasonable range (0.6-2.0), with peaks around 0.8 and 1.5

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Define transforms
def get_train_transforms():
    return transforms.Compose([
        transforms.Resize((256, 256)),  # Slightly larger for random crop
        transforms.RandomCrop(224),     # VGG16 input size
        transforms.RandomHorizontalFlip(p=0.5),  # Birds look similar horizontally
        transforms.RandomRotation(15),  # Slight rotation only
        transforms.ColorJitter(
            brightness=0.2,  # Subtle brightness changes
            contrast=0.2,    # Subtle contrast changes
            saturation=0.2,  # Careful with saturation as birds have distinctive colors
            hue=0.1         # Very minimal hue changes
        ),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # ImageNet statistics
            std=[0.229, 0.224, 0.225]
        )
    ])

def get_val_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),  # Clean resize for validation
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

# Read metadata files if not already loaded
train_df = pd.read_csv('BDMA7_project_files/train_metadata.csv')
val_df = pd.read_csv('BDMA7_project_files/val_metadata.csv')



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms

def show_augmentations(dataset, idx=0, num_augments=5):
    """
    Display original image alongside multiple augmented versions
    """
    plt.figure(figsize=(15, 3))

    # Get and display original image
    img_path = os.path.join(dataset.root_dir, 'train_images', dataset.df.iloc[idx]['path'])
    original_image = Image.open(img_path).convert('RGB')
    plt.subplot(1, num_augments+1, 1)
    plt.imshow(original_image)
    plt.title('Original')
    plt.axis('off')

    # Create augmentation pipeline without normalization for visualization
    augment_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2,
            hue=0.1
        )
    ])

    # Generate and display augmented versions
    for i in range(num_augments):
        # Apply augmentations
        augmented_image = augment_transform(original_image)

        # Display
        plt.subplot(1, num_augments+1, i+2)
        plt.imshow(augmented_image)
        plt.title(f'Augmented {i+1}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Display augmentations for multiple examples
print("Displaying augmentations for different bird species:")
for idx in [0, 10, 20]:
    species = train_dataset.df.iloc[idx]['class'].replace('_', ' ')
    print(f"\nSpecies: {species}")
    show_augmentations(train_dataset, idx)

### Expected Outcomes
After running this visualization, we should see:
1. Subtle color variations that preserve species characteristics
2. Slight rotations that maintain natural bird poses
3. Crops that keep the bird as the main subject
4. Consistent image quality across transformations

### Next Steps
Based on these visualizations, we can:
1. Adjust augmentation parameters if needed
2. Proceed to model setup if satisfactory
3. Add/remove specific augmentations

# VGG16 Model Implementation

### Architecture Overview
VGG16 is an excellent choice for our bird classification task because:
1. Well-understood architecture with proven performance
2. Good feature extraction capabilities
3. Relatively straightforward to modify and fine-tune

### Implementation Strategy
We'll implement VGG16 with these considerations:
1. Remove bird-related classes from ImageNet pretraining
2. Modify the classifier layer for our 20 classes
3. Set up proper transfer learning pipeline

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import VGG16_Weights

# ImageNet bird-related class indices
BIRD_CLASSES = [
    7,   # cock
    8,   # hen
    9,   # ostrich, Struthio camelus
    10,  # brambling, Fringilla montifringilla
    11,  # goldfinch, Carduelis carduelis
    12,  # house finch, linnet, Carpodacus mexicanus
    13,  # junco, snowbird
    14,  # indigo bunting, indigo finch, indigo bird, Passerina cyanea
    15,  # robin, American robin, Turdus migratorius
    16,  # bulbul
    84,  # peacock
    88,  # drake
    89,  # red-breasted merganser, Mergus serrator
    90,  # goose
    91,  # black swan, Cygnus atratus
    92,  # tusker
    93,  # echidna, spiny anteater, anteater
    94,  # platypus, duckbill, duckbilled platypus, duck-billed platypus
    127, # white stork, Ciconia ciconia
    128, # black stork, Ciconia nigra
    129, # spoonbill
    130, # flamingo
    131, # little blue heron, Egretta caerulea
    132, # American egret, great white heron, Egretta albus
    133, # bittern
    134, # crane
    135, # limpkin, Aramus pictus
    136, # European gallinule, Porphyrio porphyrio
    137, # American coot, marsh hen, mud hen, water hen, Fulica americana
    138, # bustard
    139, # ruddy turnstone, Arenaria interpres
    140, # red-backed sandpiper, dunlin, Erolia alpina
    141, # redshank, Tringa totanus
    142, # dowitcher
    143, # oystercatcher, oyster catcher
    144, # pelican
    145, # king penguin, Aptenodytes patagonica
    146, # albatross, mollymawk
]


class BirdFreeVGG16(nn.Module):
    def __init__(self, num_classes=20):
        super(BirdFreeVGG16, self).__init__()

        # Load pretrained VGG16 and get its features
        base_model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

        # Zero out bird-related knowledge
        with torch.no_grad():
            for bird_idx in BIRD_CLASSES:
                base_model.classifier[-1].weight[bird_idx].zero_()
                base_model.classifier[-1].bias[bird_idx].zero_()

        # Get the feature layers
        self.features = base_model.features
        self.avgpool = base_model.avgpool

        # Freeze all feature layers
        for param in self.features.parameters():
            param.requires_grad = False

        # Define classifier layers with smaller dimensions
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024),    # Reduced from 4096
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),            # Reduced from 2048
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(512, num_classes)       # Final layer
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BirdFreeVGG16().to(device)

# Print model structure to verify
print("\nModel Structure:")
print(model)

# Only train classifier parameters
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.0001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    verbose=True
)

print(f"\nModel initialized on: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Verify which layers are frozen
print("\nLayer Status:")
for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")



Model Structure:
BirdFreeVGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, str

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


### Architecture Analysis - Much Better WE REDUCED TRAINABLE PARAMETERS!

### Current Status
1. Total parameters: ~41M (reasonable for VGG16)
2. Trainable parameters: ~26M (only classifier)
3. Feature layers are properly frozen
4. Classifier dimensions are more appropriate

### Parameter Breakdown:
1. Features (frozen): ~14.7M parameters
2. Classifier (trainable): ~26.2M parameters
  - First layer: 25088 × 1024 = ~25.7M
  - Second layer: 1024 × 512 = ~524K
  - Final layer: 512 × 20 = ~10K

### Recommended Adjustment
We should further reduce the classifier size to prevent overfitting:

In [ ]:
class BirdFreeVGG16(nn.Module):
    def __init__(self, num_classes=20):
        super(BirdFreeVGG16, self).__init__()

        # Load pretrained VGG16 and get its features
        base_model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

        # Zero out bird-related knowledge
        with torch.no_grad():
            for bird_idx in BIRD_CLASSES:
                base_model.classifier[-1].weight[bird_idx].zero_()
                base_model.classifier[-1].bias[bird_idx].zero_()

        self.features = base_model.features
        self.avgpool = base_model.avgpool

        # Freeze feature layers
        for param in self.features.parameters():
            param.requires_grad = False

        # Smaller classifier for our dataset size
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),     # Reduced from 1024
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),             # Reduced from 512
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Training Strategy
Our initial approach will be:
1. Freeze early layers initially
2. Train only the classifier
3. Gradually unfreeze more layers as needed

### Next Steps
We can:
1. Set up the training loop
2. Implement model checkpointing
3. Add learning rate scheduling

# Training Loop Implementation

### Training Strategy
1. Split training into epochs
2. Monitor validation performance
3. Implement early stopping
4. Save best model checkpoints

In [ ]:
class BirdDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, dataset_type='train'):
        """
        Args:
            df (pandas.DataFrame): DataFrame with image paths and labels
            root_dir (string): Directory with all the images
            transform (callable, optional): Optional transform to be applied on images
            dataset_type (str): Either 'train' or 'val' to handle correct paths
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.dataset_type = dataset_type  # Save this as instance variable

        # Verify and clean data paths
        valid_samples = []
        for idx, row in df.iterrows():
            # Use dataset_type to determine correct subfolder
            full_path = os.path.join(root_dir, f'{dataset_type}_images', row['path'])
            if os.path.exists(full_path):
                valid_samples.append(idx)
            else:
                print(f"Warning: File not found: {full_path}")

        self.df = df.iloc[valid_samples].reset_index(drop=True)
        print(f"Found {len(self.df)} valid images out of {len(df)} total")

    def __getitem__(self, idx):
        # Use dataset_type when constructing path
        img_path = os.path.join(self.root_dir, f'{self.dataset_type}_images', self.df.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        label = self.df.iloc[idx]['class_idx']

        if self.transform:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.df)

# Create datasets with correct paths
train_dataset = BirdDataset(
    train_df,
    root_dir='BDMA7_project_files',
    transform=get_train_transforms(),
    dataset_type='train'
)

val_dataset = BirdDataset(
    val_df,
    root_dir='BDMA7_project_files',
    transform=get_val_transforms(),
    dataset_type='val'  # This is the key change
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)


Found 1082 valid images out of 1082 total
Found 103 valid images out of 103 total


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, patience=7):
    """
    Train the model with early stopping and model checkpointing

    Args:
        model: The neural network model
        criterion: Loss function
        optimizer: Optimizer
        scheduler: Learning rate scheduler
        num_epochs: Maximum number of epochs
        patience: Early stopping patience

    Returns:
        model: Best trained model
        best_acc: Best validation accuracy achieved
    """
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    no_improve_epochs = 0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Progress bar for batches
            pbar = tqdm(dataloader, desc=f'{phase} batches')

            # Iterate over data
            for inputs, labels in pbar:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Update progress bar
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            # Store history
            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc)
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if best accuracy
            if phase == 'val':
                scheduler.step(epoch_loss)

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                # Early stopping check
                if no_improve_epochs >= patience:
                    print(f'\nEarly stopping triggered after {epoch+1} epochs')
                    model.load_state_dict(best_model_wts)
                    return model, best_acc

    print(f'\nBest val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import VGG16_Weights
import time
import copy
from tqdm import tqdm



# First recreate our model
class BirdFreeVGG16(nn.Module):
    def __init__(self, num_classes=20):
        super(BirdFreeVGG16, self).__init__()

        # Load pretrained VGG16 and get its features
        base_model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

        # Zero out bird-related knowledge
        with torch.no_grad():
            for bird_idx in BIRD_CLASSES:
                base_model.classifier[-1].weight[bird_idx].zero_()
                base_model.classifier[-1].bias[bird_idx].zero_()

        self.features = base_model.features
        self.avgpool = base_model.avgpool

        # Freeze feature layers
        for param in self.features.parameters():
            param.requires_grad = False

        # Smaller classifier for our dataset size
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize model and move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BirdFreeVGG16().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.0001)

# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    verbose=True
)

print("Model setup complete. Starting training...")

# Now your training loop should work
model, history = train_model(model, criterion, optimizer, scheduler)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Model setup complete. Starting training...

Epoch 1/25
----------


train batches:  37%|███▋      | 100/271 [00:06<00:11, 15.52it/s, loss=2.8741]


KeyboardInterrupt: 

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def hyperparameter_grid_search(train_dataset, val_dataset, num_classes=20):
    """
    Perform systematic hyperparameter grid search for bird classification model

    Args:
    - train_dataset: Training dataset
    - val_dataset: Validation dataset
    - num_classes: Number of bird species

    Returns:
    - Best hyperparameters and corresponding model
    """
    # Define hyperparameter grid
    hyperparameter_grid = {
        'learning_rates': [ 1e-3, 5e-4, 1e-4, 5e-5],
        'batch_sizes': [16, 32],
        'dropout_rates': [0.3, 0.4],
        'num_epochs': [140]
    }

    # Tracking best results
    best_accuracy = 0
    best_hyperparameters = {}
    best_model = None

    # Comprehensive grid search
    for lr in hyperparameter_grid['learning_rates']:
        for batch_size in hyperparameter_grid['batch_sizes']:
            for dropout_rate in hyperparameter_grid['dropout_rates']:
                for num_epochs in hyperparameter_grid['num_epochs']:
                    print(f"\n--- Testing Configuration ---")
                    print(f"Learning Rate: {lr}")
                    print(f"Batch Size: {batch_size}")
                    print(f"Dropout Rate: {dropout_rate}")
                    print(f"Number of Epochs: {num_epochs}")

                    # Create data loaders with current batch size
                    train_loader = DataLoader(
                        train_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=2
                    )
                    val_loader = DataLoader(
                        val_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=2
                    )

                    # Reinitialize model with current dropout rate
                    class CustomVGG16(BirdFreeVGG16):
                        def __init__(self, num_classes=20, dropout_rate=0.5):
                            super().__init__(num_classes)
                            # Modify classifier to use custom dropout
                            self.classifier = nn.Sequential(
                                nn.Linear(512 * 7 * 7, 1024),
                                nn.ReLU(True),
                                nn.Dropout(p=dropout_rate),
                                nn.Linear(1024, 512),
                                nn.ReLU(True),
                                nn.Dropout(p=dropout_rate),
                                nn.Linear(512, num_classes)
                            )

                    model = CustomVGG16(num_classes, dropout_rate).to(device)

                    # Define optimizer with current learning rate
                    optimizer = torch.optim.Adam(
                        model.classifier.parameters(),
                        lr=lr
                    )

                    # Learning rate scheduler
                    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                        optimizer,
                        mode='min',
                        factor=0.1,
                        patience=4
                    )

                    # Training loop
                    try:
                        # Save model state before training
                        initial_state = copy.deepcopy(model.state_dict())

                        # Train model
                        trained_model, val_accuracy = train_model(
                            model,
                            criterion,
                            optimizer,
                            scheduler,
                            num_epochs=num_epochs
                        )

                        # Update best configuration if improved
                        if val_accuracy > best_accuracy:
                            best_accuracy = val_accuracy
                            best_hyperparameters = {
                                'learning_rate': lr,
                                'batch_size': batch_size,
                                'dropout_rate': dropout_rate,
                                'num_epochs': num_epochs
                            }
                            best_model = trained_model

                        print(f"Validation Accuracy: {val_accuracy:.2f}%")

                    except Exception as e:
                        print(f"Error in training: {e}")
                        # Restore model to initial state
                        model.load_state_dict(initial_state)

    # Final results
    print("\n--- Best Hyperparameters ---")
    for key, value in best_hyperparameters.items():
        print(f"{key}: {value}")
    print(f"Best Validation Accuracy: {best_accuracy:.2f}%")

    return best_model, best_hyperparameters, best_accuracy

# Run hyperparameter grid search
best_model, best_hyperparameters, best_accuracy = hyperparameter_grid_search(
    train_dataset,
    val_dataset
)

# Optional: Save best model
torch.save({
    'model_state_dict': best_model.state_dict(),
    'hyperparameters': best_hyperparameters,
    'accuracy': best_accuracy
}, '/content/drive/MyDrive/Aimen/best_bird_classifier.pth')


--- Testing Configuration ---
Learning Rate: 0.001
Batch Size: 16
Dropout Rate: 0.3
Number of Epochs: 140

Epoch 1/140
----------


train batches: 100%|██████████| 271/271 [00:15<00:00, 18.04it/s, loss=3.0581]


train Loss: 2.6541 Acc: 0.2431


val batches: 100%|██████████| 7/7 [00:00<00:00,  8.11it/s, loss=1.7310]


val Loss: 2.5938 Acc: 0.3786

Epoch 2/140
----------


...

val Loss: 1.5011 Acc: 0.6893

Epoch 24/140
----------


train batches: 100%|██████████| 271/271 [00:16<00:00, 16.59it/s, loss=0.0017]


train Loss: 0.0059 Acc: 1.0000


val batches: 100%|██████████| 7/7 [00:01<00:00,  5.80it/s, loss=0.9172]


val Loss: 1.5016 Acc: 0.6796

Early stopping triggered after 24 epochs
Validation Accuracy: 0.71%

--- Best Hyperparameters ---
learning_rate: 0.0001
batch_size: 16
dropout_rate: 0.3
num_epochs: 140
Best Validation Accuracy: 0.73%


### Key Components:
1. Progress bars with tqdm
2. Early stopping mechanism
3. Model checkpointing
4. Learning rate scheduling
5. Training history tracking

### Next Steps:
1. Visualize training progress
2. Analyze model performance
3. Make adjustments if needed